In [62]:
import yaml
import os
import papermill as pm
import math

In [63]:
workflow_name = "compare_two_CORE_ollie"

In [64]:
with open(f'./{workflow_name}.yml') as file:
    settings = yaml.load(file, Loader=yaml.FullLoader)
settings['years'] = list(range(settings['start_year'], settings['end_year']+1))

In [65]:
ofolder = f"./results/{workflow_name}"
ofolder_notebooks = os.path.join(ofolder, "notebooks")
ofolder_figures = os.path.join(ofolder, "figures")
if not os.path.exists(ofolder):
    os.makedirs(ofolder_notebooks)
    os.makedirs(ofolder_figures)

In [66]:
settings

{'meshpath': '/Users/nkolduno/PYTHON/DATA/core2_meanz_mesh/',
 'abg': [50, 15, -90],
 'climatology_path': '/Users/nkolduno/PYTHON/DATA/CLIM_on_mesh/core2_meanz/',
 'climatology_year': 1948,
 'columns': 2,
 'bbox': [-180, 180, -80, 90],
 'res': [360, 180],
 'mapproj': 'pc',
 'figsize': [24, 14],
 'figsize_small': [7, 4.326],
 'start_year': 1978,
 'end_year': 1986,
 'difference': {'temp': {'levels': [-1, 1, 41], 'depths': [0, 100, 1000]},
  'salt': {'levels': [-0.5, 0.5, 41],
   'depths': [0, 100, 1000],
   'units': 'psu'}},
 'climatology': {'temp': {'levels': [-5, 5, 41], 'depths': [0, 100, 1000]},
  'salt': {'levels': [-1, 1, 41], 'depths': [0, 100, 1000], 'units': 'psu'}},
 'ice_integrals': {'figsize': [10, 5], 'hemisphere': 'N'},
 'hovm_difference': {'Eurasian basin': {'temp': {'levels': [-0.1, 0.1, 41],
    'maxdepth': 1000},
   'salt': {'levels': [-0.1, 0.1, 41], 'maxdepth': 1000}},
  'Amerasian basin': {'temp': {'levels': [-0.1, 0.1, 41], 'maxdepth': 1000},
   'salt': {'levels': [

In [67]:
input_paths = ['/Users/nkolduno/PYTHON/DATA/output_7_8/', 
               "/Users/nkolduno/PYTHON/DATA/output_7_10"]
input_names = None

In [68]:
if input_names is None:
    input_names = []
    for run in input_paths:
        run = os.path.join(run, '')
        input_names.append(run.split('/')[-2])
elif len(input_names) != len(input_paths):
    raise ValueError('The size of input_names is not equal to the size of input_paths')

In [39]:
def define_rowscol(number_paths, columns):
    if number_paths < columns:
        ncol = number_paths
    else:
        ncol = columns
    nrows = math.ceil(number_paths / columns)
    return [nrows, ncol]

In [125]:
def drive_difference(difference, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures):
    number_paths = len(input_paths[1:])
    for variable in difference:
        for depth in difference[variable]['depths']:
            current_params = settings.copy()
            current_params['input_paths'] = input_paths[1:]
            current_params['input_names'] = input_names[1:]
            current_params['reference_path'] = input_paths[0]
            current_params['reference_name'] = input_names[0]
            current_params['reference_years'] = settings['years']
            current_params['variable'] = variable
            current_params['depth'] = depth
            current_params['levels'] = difference[variable]['levels']
            current_params['rowscol'] = define_rowscol(number_paths, settings['columns'])
            if 'units' in difference[variable]:
                current_params['units'] =  difference[variable]['units']
            current_params['ofile'] = f"{ofolder_figures}/{workflow_name}_difference_{variable}_{depth}.png"
            pm.execute_notebook(
                          './templates/difference.ipynb',
                          f'{ofolder_notebooks}/{workflow_name}_difference_{variable}_{depth}.ipynb',
            parameters=current_params, nest_asyncio=True)

In [126]:
if 'difference' in settings:
    difference = settings['difference'].copy()
    number_paths = len(input_paths)
    if number_paths<2:
        raise ValueError('You specified `difference` analysis, but provide only one path')
    drive_difference(difference, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures)

In [103]:
def drive_ice_integrals(ice_integrals, settings):
    current_params = settings.copy()
    current_params['input_paths'] = input_paths
    current_params['input_names'] = input_names
    current_params['hemisphere'] = ice_integrals['hemisphere']
    if 'figsize' in ice_integrals:
        current_params['figsize'] = ice_integrals['figsize']
    current_params['ofile'] = f"{ofolder_figures}/{workflow_name}_ice_integrals"
    pm.execute_notebook('./templates/ice_integrals.ipynb',
                          f'{ofolder_notebooks}/{workflow_name}_ice_integrals.ipynb',
            parameters=current_params, nest_asyncio=True)

In [104]:
if 'ice_integrals' in settings:
    ice_integrals = settings['ice_integrals'].copy()
    drive_ice_integrals(ice_integrals, settings)

In [93]:
def drive_climatology(climatology, settings, input_paths, input_names):
    number_paths = len(input_paths)
    for variable in climatology:
        for depth in climatology[variable]['depths']:
            current_params = settings.copy()
            current_params['input_paths'] = input_paths
            current_params['input_names'] = input_names
            current_params['reference_path'] = settings['climatology_path']
            current_params['reference_name'] = 'clim'
            current_params['reference_years'] = settings['climatology_year']
            current_params['variable'] = variable
            current_params['depth'] = depth
            current_params['levels'] = climatology[variable]['levels']
            current_params['rowscol'] = define_rowscol(number_paths, settings['columns'])
            if 'units' in climatology[variable]:
                current_params['units'] =  climatology[variable]['units']
            current_params['ofile'] = f"{ofolder_figures}/{workflow_name}_climatology_{variable}_{depth}.png"
            pm.execute_notebook(
                          './templates/difference.ipynb',
                          f'{ofolder_notebooks}/{workflow_name}_climatology_{variable}_{depth}.ipynb',
            parameters=current_params, nest_asyncio=True)

In [94]:
if 'climatology' in settings:
    climatology = settings['climatology'].copy()
    drive_climatology(climatology, settings, input_paths, input_names)

In [110]:
def drive_difference_hovm(difference, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures):
    number_paths = len(input_paths[1:])
    for region_name, region in hovm_difference.items():
#     for variable in difference:
      for variable_name, variable in region.items():
        print(variable['levels'])
        current_params = settings.copy()
        current_params['input_paths'] = input_paths[1:]
        current_params['input_names'] = input_names[1:]
        current_params['reference_path'] = input_paths[0]
        current_params['reference_name'] = input_names[0]
        current_params['reference_years'] = settings['years']
        current_params['variable'] = variable_name
        current_params['levels'] = variable['levels']
        current_params['maxdepth'] = variable['maxdepth']
        current_params['region'] = region_name
        if 'units' in variable:
            current_params['units'] =  variable['units']
        region_name_underscore = region_name.replace(' ', '_')
        current_params['ofile'] = f"{ofolder_figures}/{workflow_name}_hovm_difference_{region_name_underscore}_{variable_name}.png"
        
        print(current_params['ofile'])
        pm.execute_notebook(
                      './templates/hovm_difference.ipynb',
                      f'{ofolder_notebooks}/{workflow_name}_hovm_difference_{region_name_underscore}_{variable_name}.ipynb',
        parameters=current_params, nest_asyncio=True)

In [111]:
hovm_difference = settings['hovm_difference'].copy()

In [112]:
for region_name, region in hovm_difference.items():
    for variable_name, variable in region.items():
        print(region_name)
        print(variable_name)
        print(variable['levels'])
#     print(key)
#     print(value)

Eurasian basin
temp
[-0.1, 0.1, 41]
Eurasian basin
salt
[-0.1, 0.1, 41]
Amerasian basin
temp
[-0.1, 0.1, 41]
Amerasian basin
salt
[-0.1, 0.1, 41]


In [113]:
variable

{'levels': [-0.1, 0.1, 41], 'maxdepth': 1000}

In [114]:
if 'hovm_difference' in settings:
    hovm_difference = settings['hovm_difference'].copy()
    number_paths = len(input_paths)
    if number_paths<2:
        raise ValueError('You specified `difference` analysis, but provide only one path')
    drive_difference_hovm(hovm_difference, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures)

[-0.1, 0.1, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_hovm_difference_Eurasian_basin_temp.png



[-0.1, 0.1, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_hovm_difference_Eurasian_basin_salt.png



[-0.1, 0.1, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_hovm_difference_Amerasian_basin_temp.png



[-0.1, 0.1, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_hovm_difference_Amerasian_basin_salt.png


In [137]:
def drive_difference_hovm_clim(difference, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures):
    number_paths = len(input_paths[1:])
    for region_name, region in hovm_difference.items():
#     for variable in difference:
      for variable_name, variable in region.items():
        print(variable['levels'])
        current_params = settings.copy()
        current_params['input_paths'] = input_paths[:]
        current_params['input_names'] = input_names[:]
        current_params['reference_path'] = settings['climatology_path']
        current_params['reference_name'] = 'clim'
        current_params['reference_years'] = settings['climatology_year']
        current_params['variable'] = variable_name
        current_params['levels'] = variable['levels']
        current_params['maxdepth'] = variable['maxdepth']
        current_params['region'] = region_name
        if 'units' in variable:
            current_params['units'] =  variable['units']
        region_name_underscore = region_name.replace(' ', '_')
        current_params['ofile'] = f"{ofolder_figures}/{workflow_name}_hovm_difference_clim_{region_name_underscore}_{variable_name}.png"
        
        print(current_params['ofile'])
        pm.execute_notebook(
                      './templates/hovm_difference.ipynb',
                      f'{ofolder_notebooks}/{workflow_name}_hovm_difference_clim_{region_name_underscore}_{variable_name}.ipynb',
        parameters=current_params, nest_asyncio=True)

In [149]:
if 'hovm_difference_clim' in settings:
    hovm_difference = settings['hovm_difference_clim'].copy()
    number_paths = len(input_paths)
    if number_paths<2:
        raise ValueError('You specified `difference` analysis, but provide only one path')
    drive_difference_hovm_clim(hovm_difference, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures)

[-0.3, 0.3, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_hovm_difference_clim_Eurasian_basin_temp.png



[-0.1, 0.1, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_hovm_difference_clim_Eurasian_basin_salt.png



[-0.3, 0.3, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_hovm_difference_clim_Amerasian_basin_temp.png



[-0.1, 0.1, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_hovm_difference_clim_Amerasian_basin_salt.png



[-0.1, 0.1, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_hovm_difference_clim_Global_Ocean_temp.png



[-0.1, 0.1, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_hovm_difference_clim_Global_Ocean_salt.png


In [192]:
def drive_ocean_integrals(oce_integrals, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures):
    number_paths = len(input_paths[1:])
    for region_name, region in oce_integrals.items():
#     for variable in difference:
      for variable_name, variable in region.items():
        for uplow in variable['uplows']:
#         print(variable['levels'])
            current_params = settings.copy()
            current_params['input_paths'] = input_paths[:]
            current_params['input_names'] = input_names[:]
    #         current_params['reference_path'] = input_paths[0]
    #         current_params['reference_name'] = input_names[0]
    #         current_params['reference_years'] = settings['years']
            if 'figsize_small' in settings:
                current_params['figsize'] = settings['figsize_small']
                print(current_params['figsize'])
            current_params['variable'] = variable_name
            current_params['uplow'] = uplow
            print(uplow)
            current_params['region'] = region_name
            region_name_underscore = region_name.replace(' ', '_')
            current_params['ofile'] = f"{ofolder_figures}/{workflow_name}_oce_integrals_{region_name_underscore}_{variable_name}_{uplow[0]}_{uplow[1]}.png"

            print(current_params['ofile'])
            pm.execute_notebook(
                          './templates/ocean_integrals.ipynb',
                          f'{ofolder_notebooks}/{workflow_name}_oce_integrals_{region_name_underscore}_{variable_name}_{uplow[0]}_{uplow[1]}.ipynb',
            parameters=current_params, nest_asyncio=True)

In [194]:
if 'oce_integrals' in settings:
    oce_integrals = settings['oce_integrals'].copy()
#     number_paths = len(input_paths)
#     if number_paths<2:
#         raise ValueError('You specified `difference` analysis, but provide only one path')
    drive_ocean_integrals(oce_integrals, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures)

[7, 4.326]
[0, 100]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_oce_integrals_Global_Ocean_temp_0_100.png



[7, 4.326]
[0, 700]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_oce_integrals_Global_Ocean_temp_0_700.png



[7, 4.326]
[0, 100]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_oce_integrals_Global_Ocean_salt_0_100.png



[7, 4.326]
[0, 700]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_oce_integrals_Global_Ocean_salt_0_700.png



[7, 4.326]
[0, 100]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_oce_integrals_Eurasian_basin_temp_0_100.png



[7, 4.326]
[0, 700]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_oce_integrals_Eurasian_basin_temp_0_700.png



[7, 4.326]
[0, 100]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_oce_integrals_Eurasian_basin_salt_0_100.png



[7, 4.326]
[0, 700]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_oce_integrals_Eurasian_basin_salt_0_700.png


In [158]:
settings['oce_integrals']['Global Ocean']['temp']['uplows'][0]

[0, 100]

In [40]:
def drive_xmoc(xmoc, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures):
    number_paths = len(input_paths[1:])
    for region_name, region in xmoc.items():
#     for variable in difference:
#       for variable_name, variable in region.items():
#         for uplow in variable['uplows']:
#         print(variable['levels'])
        current_params = settings.copy()
        current_params['input_paths'] = input_paths[:]
        current_params['input_names'] = input_names[:]
#         current_params['reference_path'] = input_paths[0]
#         current_params['reference_name'] = input_names[0]
#         current_params['reference_years'] = settings['years']
        if 'figsize_small' in settings:
            current_params['figsize'] = settings['figsize_small']
#             print(current_params['figsize'])
#         current_params['variable'] = variable_name
#         current_params['uplow'] = uplow
#         print(uplow)
        current_params['region'] = region_name
        current_params['levels'] = region['levels']
        print(current_params['levels'] )
        region_name_underscore = region_name.replace(' ', '_')
        current_params['ofile'] = f"{ofolder_figures}/{workflow_name}_xmoc_{region_name_underscore}.png"

        print(current_params['ofile'])
        pm.execute_notebook(
                      './templates/xmoc.ipynb',
                      f'{ofolder_notebooks}/{workflow_name}_xmoc_{region_name_underscore}.ipynb',
        parameters=current_params, nest_asyncio=True)

In [41]:
if 'xmoc' in settings:
    xmoc = settings['xmoc'].copy()
#     number_paths = len(input_paths)
#     if number_paths<2:
#         raise ValueError('You specified `difference` analysis, but provide only one path')
    drive_xmoc(xmoc, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures)

[-20, 20, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_xmoc_Global_Ocean.png



[-5, 15, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_xmoc_Atlantic_MOC.png


In [60]:
def drive_xmoc_difference(xmoc, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures):
    number_paths = len(input_paths[1:])
    for region_name, region in xmoc.items():
#     for variable in difference:
#       for variable_name, variable in region.items():
#         for uplow in variable['uplows']:
#         print(variable['levels'])
        current_params = settings.copy()
        current_params['input_paths'] = input_paths[1:]
        current_params['input_names'] = input_names[1:]
        current_params['reference_path'] = input_paths[0]
        current_params['reference_name'] = input_names[0]
        current_params['reference_years'] = settings['years']
        if 'figsize_small' in settings:
            current_params['figsize'] = settings['figsize_small']
#             print(current_params['figsize'])
#         current_params['variable'] = variable_name
#         current_params['uplow'] = uplow
#         print(uplow)
        current_params['region'] = region_name
        current_params['levels'] = region['levels']
        print(current_params['levels'] )
        region_name_underscore = region_name.replace(' ', '_')
        current_params['ofile'] = f"{ofolder_figures}/{workflow_name}_xmoc_difference_{region_name_underscore}.png"

        print(current_params['ofile'])
        pm.execute_notebook(
                      './templates/xmoc_difference.ipynb',
                      f'{ofolder_notebooks}/{workflow_name}_xmoc_difference_{region_name_underscore}.ipynb',
        parameters=current_params, nest_asyncio=True)

In [61]:
if 'xmoc_difference' in settings:
    xmoc_difference = settings['xmoc_difference'].copy()
    number_paths = len(input_paths)
    if number_paths<2:
        raise ValueError('You specified `difference` analysis, but provide only one path')
    drive_xmoc_difference(xmoc_difference, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures)

[-5, 5, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_xmoc_difference_Global_Ocean.png



[-2, 2, 41]
./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_xmoc_difference_Atlantic_MOC.png


In [82]:
def drive_amoc_timeseries(amoc_timeseries, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures):
    number_paths = len(input_paths[1:])
#     for region_name, region in xmoc.items():
#     for variable in difference:
#       for variable_name, variable in region.items():
#         for uplow in variable['uplows']:
#         print(variable['levels'])
    current_params = settings.copy()
    current_params['input_paths'] = input_paths[:]
    current_params['input_names'] = input_names[:]
    current_params['latitudes'] = amoc_timeseries['latitudes']
#     current_params['reference_path'] = input_paths[0]
#     current_params['reference_name'] = input_names[0]
#     current_params['reference_years'] = settings['years']
    if 'figsize_small' in settings:
        current_params['figsize'] = settings['figsize_small']
#             print(current_params['figsize'])
#         current_params['variable'] = variable_name
#         current_params['uplow'] = uplow
#         print(uplow)
#         current_params['region'] = region_name
#         current_params['levels'] = region['levels']
#         print(current_params['levels'] )
#     region_name_underscore = region_name.replace(' ', '_')
    current_params['ofile'] = f"{ofolder_figures}/{workflow_name}_amoc_timeseries"

    print(current_params['ofile'])
    pm.execute_notebook(
                  './templates/amoc_timeseries.ipynb',
                  f'{ofolder_notebooks}/{workflow_name}_amoc_timeseries.ipynb',
    parameters=current_params, nest_asyncio=True)

In [83]:
if 'amoc_timeseries' in settings:
    amoc_timeseries = settings['amoc_timeseries'].copy()
#     number_paths = len(input_paths)
#     if number_paths<2:
#         raise ValueError('You specified `difference` analysis, but provide only one path')
    drive_amoc_timeseries(amoc_timeseries, settings, input_paths, input_names, ofolder_notebooks, ofolder_figures)

./results/compare_two_CORE_ollie/figures/compare_two_CORE_ollie_amoc_timeseries


In [10]:
try:
    del params['difference']
except:
    pass

In [11]:
params['input_paths'] = input_paths

In [17]:
pm.execute_notebook(
        './templates/difference.ipynb',
        't2.ipynb',
         parameters=settings, nest_asyncio=True)

{'cells': [{'cell_type': 'code',
   'execution_count': 1,
   'metadata': {'tags': [],
    'papermill': {'exception': False,
     'start_time': '2020-05-19T08:55:55.029538',
     'end_time': '2020-05-19T08:55:57.369185',
     'duration': 2.339647,
     'status': 'completed'},
    'execution': {'iopub.status.busy': '2020-05-19T08:55:55.043090Z',
     'iopub.execute_input': '2020-05-19T08:55:55.043594Z',
     'iopub.status.idle': '2020-05-19T08:55:57.368454Z',
     'shell.execute_reply': '2020-05-19T08:55:57.369036Z'}},
   'outputs': [{'output_type': 'stream',
     'name': 'stdout',
     'text': 'osgeo is not installed, conversion to Geo formats like Geotiff (fesom2GeoFormat) will not work.\n'}],
   'source': 'import pyfesom2 as pf\nimport xarray as xr\nimport os\nimport pandas as pd\nfrom collections import OrderedDict\nfrom itertools import product\nimport matplotlib.pylab as plt\n%matplotlib inline'},
  {'cell_type': 'code',
   'execution_count': 2,
   'metadata': {'tags': ['parameters